# OCR 文字识别测试

基于 RapidOCR 的文字识别演示，支持图片上传、文字检测和识别结果可视化。

## 1. 安装依赖

In [ ]:
!pip install -q rapidocr-onnxruntime gradio opencv-python-headless pillow

## 2. 导入必要的库

In [ ]:
import gradio as gr
from rapidocr_onnxruntime import RapidOCR
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

## 3. 定义绘制检测框的函数

In [ ]:
def draw_boxes(image, boxes, txts, scores):
    if isinstance(image, np.ndarray):
        img = image.copy()
    else:
        img = np.array(image)
    
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    elif img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    pil_img = Image.fromarray(img)
    overlay = Image.new('RGBA', pil_img.size, (255, 255, 255, 0))
    draw = ImageDraw.Draw(overlay)
    
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 20)
        small_font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 16)
    except:
        try:
            font = ImageFont.truetype("arial.ttf", 20)
            small_font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()
            small_font = ImageFont.load_default()
    
    colors = [
        (255, 99, 71), (30, 144, 255), (50, 205, 50), (255, 165, 0),
        (147, 112, 219), (255, 20, 147), (0, 191, 255), (255, 215, 0)
    ]
    
    for idx, (box, txt, score) in enumerate(zip(boxes, txts, scores)):
        box = box.astype(np.int32)
        points = [(int(box[i][0]), int(box[i][1])) for i in range(len(box))]
        
        color = colors[idx % len(colors)]
        fill_color = (*color, 60)
        draw.polygon(points, fill=fill_color, outline=(*color, 200))
        
        min_x = int(np.min(box[:, 0]))
        min_y = int(np.min(box[:, 1]))
        
        text_label = f"{txt} ({score:.2f})"
        bbox = draw.textbbox((0, 0), text_label, font=small_font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        
        label_y = max(min_y - text_height - 5, 0)
        draw.rectangle(
            [(min_x, label_y), (min_x + text_width + 10, label_y + text_height + 5)],
            fill=(*color, 200)
        )
        draw.text((min_x + 5, label_y), text_label, fill=(255, 255, 255, 255), font=small_font)
    
    pil_img = Image.alpha_composite(pil_img.convert('RGBA'), overlay)
    result = np.array(pil_img.convert('RGB'))
    
    return result

## 4. 定义 OCR 处理函数

In [ ]:
def ocr_image(image):
    if image is None:
        return None, "请上传图片", None, ""
    
    engine = RapidOCR()
    result, elapse = engine(image)
    
    if result is None or len(result) == 0:
        return image, "未检测到文字", None, ""
    
    boxes = [line[0] for line in result]
    txts = [line[1] for line in result]
    scores = [line[2] for line in result]
    
    vis_image = draw_boxes(image, np.array(boxes), txts, scores)
    
    full_text = "\n".join(txts)
    
    detailed_result = []
    for i, (text, score) in enumerate(zip(txts, scores), 1):
        detailed_result.append(f"[{i}] {text} | 置信度: {score:.4f}")
    
    detailed_output = "\n".join(detailed_result)
    
    elapse_info = f"总耗时: {elapse:.3f}秒"
    
    return vis_image, full_text, detailed_output, elapse_info

## 5. 创建并启动 Gradio 界面

In [ ]:
with gr.Blocks(title="OCR 测试验证") as demo:
    gr.Markdown("# OCR 文字识别测试")
    gr.Markdown("上传图片进行文字识别，查看检测框和识别结果")
    
    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(type="numpy", label="上传图片")
            submit_btn = gr.Button("开始识别", variant="primary", size="lg")
        
        with gr.Column(scale=1):
            image_output = gr.Image(type="numpy", label="识别结果可视化（带边框）")
    
    with gr.Row():
        with gr.Column():
            text_output = gr.Textbox(label="识别文本", lines=10)
        
        with gr.Column():
            detail_output = gr.Textbox(label="详细信息（含置信度）", lines=10)
    
    with gr.Row():
        elapse_output = gr.Textbox(label="识别耗时", lines=2)
    
    submit_btn.click(
        fn=ocr_image,
        inputs=[image_input],
        outputs=[image_output, text_output, detail_output, elapse_output]
    )

demo.launch(share=True, debug=True)